In [0]:
import time

source_path = "/Volumes/workspace/bronze/staging/sales/"
checkpoint_path = "/Volumes/workspace/bronze/staging/sales_checkpoints"

df_stream = (spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "json")
  .option("cloudFiles.schemaLocation", checkpoint_path)
  .load(source_path)
)

while True:
  print("Iniciando una nueva ejecución para procesar archivos disponibles...")
  
  (df_stream.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path + "_delta")
    .trigger(availableNow=True)
    .table("mi_tabla_procesada")
    .awaitTermination() 
  )
  
  print(f"Ejecución completada. Esperando 30 segundos...")
  time.sleep(30) 

In [0]:
%sql
select * from mi_tabla_procesada